In [1]:
import pandas as pd 
import requests
import config

In [2]:
API_KEY = config.api_key
response_list = []

In [3]:
for movie_id in range(550,556):
    url = 'https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, API_KEY)
    r = requests.get(url)
    response_list.append(r.json())

In [6]:
df = pd.DataFrame.from_dict(response_list)
df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/rr7E0NoGKxvbkb89eR1GwfoYjpA.jpg,None,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,...,1999-10-15,100853753,139,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.4,24186
1,False,/v1QEIuBM1vvpvfqalahhIyXY0Cm.jpg,"{'id': 372257, 'name': 'The Poseidon Adventure...",5000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",,551,tt0069113,en,The Poseidon Adventure,...,1972-12-13,84563118,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"Hell, Upside Down",The Poseidon Adventure,False,7.2,647
2,False,/k4JIHyAXaGHwAwT7y5Skd17f0Wl.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,552,tt0237539,it,Pane e tulipani,...,2000-03-03,8478434,114,"[{'english_name': 'Italian', 'iso_639_1': 'it'...",Released,Imagine your life. Now go live it.,Bread and Tulips,False,7.3,208
3,False,/r3xsFBD1VTUusk393bBc7SsDUJe.jpg,"{'id': 1952, 'name': 'USA: Land of Opportuniti...",10000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",,553,tt0276919,en,Dogville,...,2003-05-19,16680836,178,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A quiet little town not far from here.,Dogville,False,7.8,1934
4,False,/1qwXItFKqvKYyW1CwbYhxyUC8Pj.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,554,tt0308476,ru,Кукушка,...,2002-01-01,0,100,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,She's Making Peace One Man at a Time.,The Cuckoo,False,7.1,65
5,False,None,None,0,"[{'id': 53, 'name': 'Thriller'}]",http://www.luecke-im-system.de/,555,tt0442896,en,Absolut,...,2005-04-20,0,94,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,,Absolut,False,7.9,20


In [9]:
df_columns = ['budget', 'genres', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']
df['genres']

0                        [{'id': 18, 'name': 'Drama'}]
1    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
2    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
3    [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
4    [{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...
5                     [{'id': 53, 'name': 'Thriller'}]
Name: genres, dtype: object

This column is a mess. 

It’s a column of lists of JSON records, which is hard to read or quickly understand in this format. We want to expand this column out so we can easily see and make use of the internal records.

One way to do this is by exploding out the column of lists into one-hot categorical columns. This is done by creating a single column for each categorical value and setting the row value to 1 if the movie belongs to that category and 0 if it doesn’t.

The 'name' property is the part of the genre we care about. 

In [11]:
genres_list = df['genres'].tolist()
flat_list = [item for sublist in genres_list for item in sublist]

In [14]:
result = []
for l in genres_list:
    r = []
    for d in l:
        r.append(d['name'])
    result.append(r)
df = df.assign(genres_all=result)

In [16]:
#creating the genre table
df_genres = pd.DataFrame.from_records(flat_list).drop_duplicates()
df_genres

,id,name
0,18,Drama
1,28,Action
2,12,Adventure
3,53,Thriller
4,35,Comedy
5,10749,Romance
6,80,Crime
10,36,History


In [17]:
df[df_columns]

,budget,genres,id,imdb_id,original_title,release_date,revenue,runtime
0,63000000,"[{'id': 18, 'name': 'Drama'}]",550,tt0137523,Fight Club,1999-10-15,100853753,139
1,5000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",551,tt0069113,The Poseidon Adventure,1972-12-13,84563118,117
2,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",552,tt0237539,Pane e tulipani,2000-03-03,8478434,114
3,10000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",553,tt0276919,Dogville,2003-05-19,16680836,178
4,0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",554,tt0308476,Кукушка,2002-01-01,0,100
5,0,"[{'id': 53, 'name': 'Thriller'}]",555,tt0442896,Absolut,2005-04-20,0,94


In [18]:
"""
This gives us a table of the genre properties name and id. We attach the list of genre names onto our
df_columns list as shown below and remove the original genres column from the list.

The last two lines are where we use the explode() and crosstab() functions to create
the genre columns and join them onto the main table. This takes a column of lists
and turns it into a set of columns of frequency values.
"""

df_columns = ['budget', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']
df_genre_columns = df_genres['name'].to_list()
df_columns.extend(df_genre_columns)

s = df['genres_all'].explode()
df = df.join(pd.crosstab(s.index, s))

In [19]:
df[df_columns]

,budget,id,imdb_id,original_title,release_date,revenue,runtime,Drama,Action,Adventure,Thriller,Comedy,Romance,Crime,History
0,63000000,550,tt0137523,Fight Club,1999-10-15,100853753,139,1,0,0,0,0,0,0,0
1,5000000,551,tt0069113,The Poseidon Adventure,1972-12-13,84563118,117,0,1,1,1,0,0,0,0
2,0,552,tt0237539,Pane e tulipani,2000-03-03,8478434,114,0,0,0,0,1,1,0,0
3,10000000,553,tt0276919,Dogville,2003-05-19,16680836,178,1,0,0,1,0,0,1,0
4,0,554,tt0308476,Кукушка,2002-01-01,0,100,1,0,0,0,1,1,0,1
5,0,555,tt0442896,Absolut,2005-04-20,0,94,0,0,0,1,0,0,0,0


In [23]:
#dealing with release dates
df['release_date'] = pd.to_datetime(df['release_date'])
df['day'] = df['release_date'].dt.day
df['month'] = df['release_date'].dt.month
df['year'] = df['release_date'].dt.year
df['day_of_week'] = df['release_date'].dt.day_name()
df_time_columns = ['id', 'release_date', 'day', 'month', 'year', 'day_of_week']
df[df_time_columns]
df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,Comedy,Crime,Drama,History,Romance,Thriller,day,month,year,day_of_week
0,False,/rr7E0NoGKxvbkb89eR1GwfoYjpA.jpg,None,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,...,0,0,1,0,0,0,15,10,1999,Friday
1,False,/v1QEIuBM1vvpvfqalahhIyXY0Cm.jpg,"{'id': 372257, 'name': 'The Poseidon Adventure...",5000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",,551,tt0069113,en,The Poseidon Adventure,...,0,0,0,0,0,1,13,12,1972,Wednesday
2,False,/k4JIHyAXaGHwAwT7y5Skd17f0Wl.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,552,tt0237539,it,Pane e tulipani,...,1,0,0,0,1,0,3,3,2000,Friday
3,False,/r3xsFBD1VTUusk393bBc7SsDUJe.jpg,"{'id': 1952, 'name': 'USA: Land of Opportuniti...",10000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",,553,tt0276919,en,Dogville,...,0,1,1,0,0,1,19,5,2003,Monday
4,False,/1qwXItFKqvKYyW1CwbYhxyUC8Pj.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,554,tt0308476,ru,Кукушка,...,1,0,1,1,1,0,1,1,2002,Tuesday
5,False,None,None,0,"[{'id': 53, 'name': 'Thriller'}]",http://www.luecke-im-system.de/,555,tt0442896,en,Absolut,...,0,0,0,0,0,1,20,4,2005,Wednesday


In [24]:
#loading, the L in ETL
df.to_csv('tmdb_total.csv', index=False)
df[df_columns].to_csv('tmdb_movies.csv', index=False)
df_genres.to_csv('tmdb_genres.csv', index=False)
df[df_time_columns].to_csv('tmdb_datetimes.csv', index=False)